## 1. 데이터 준비

In [1]:
%pip install influxdb_client

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import zscore
from datetime import datetime, timedelta
from influxdb_client import InfluxDBClient

# 한글 폰트 설정
mpl.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False

## Influx DB 연결 정보 및 장비 위치 매핑
InfluxDB에 대한 연결 매개변수를 설정하고 장치 ID를 해당 위치에 매핑하여 데이터 검색을 구성합니다.

In [3]:
# InfluxDB 설정 정보
url = "http://133.186.144.22:8086"
token = "BPJ1pnKvoaov4Tte971t0zpRSTUXNZvrshU7u3UPheAIsBeUJEFfbKjfsZjtwZmugkHJEGRW17lH4bR9ybanNQ=="
org = "smoothing"

# 디바이스 ID와 위치를 매핑
location_mapping = {
    '24e124126d152919': 'indoor',
    '24e124126d152969': 'bottom_right_corner',
    '24e124128c067999': 'indoor',
    '24e124785c389818': 'bottom_left_corner',
    '24e124785c421885': 'top_right_corner'
}

## 데이터 가져오기
사전에 정의된 쿼리를 사용하여 InfluxDB에서 데이터를 검색합니다.

In [4]:
# InfluxDB 클라이언트 생성
def create_client(url, token, org):
    return InfluxDBClient(url=url, token=token, org=org)

# 쿼리 실행 및 DataFrame으로 변환
def query_to_dataframe(client, query):
    result = client.query_api().query(query=query)
    results = []

    for table in result:
        for record in table.records:
            results.append({
                "time": record.get_time(),
                "value": record.get_value(),
                "place": record.values.get("place"),
                "location": record.values.get("location"),
                "device": record.values.get("device")
            })

    df = pd.DataFrame(results)
    df['time'] = df['time'].astype(str).str.replace(r'\+00:00$', '', regex=True)
    return df

# 데이터를 날짜를 지정하여 CSV 파일로 저장
def save_csv(df, file_pattern, directory):
    # 경로가 존재하는지 확인하고, 없다면 생성
    if not os.path.exists(directory):
        os.makedirs(directory)

    current_date = datetime.now()
    previous_date = current_date - timedelta(days=1)
    filename = f"{directory}{previous_date.strftime(file_pattern)}"
    df.to_csv(filename, index=False)

# Data에서 'device' 열에 따라 'location' 열을 업데이트    
def update_location(df, location_mapping):
    df['location'] = df['device'].map(location_mapping)
    return df

### 데이터 조회 및 CSV 변환 수행

In [6]:
# 클라이언트 생성 및 쿼리 실행
client = create_client(url, token, org)

# 전력 조회 Flux 쿼리
query_power_usage_data = '''
import "date"
from(bucket: "powermetrics_data")
  |> range(start: date.sub(d: 1d, from: date.truncate(t: now(), unit: 1d)), stop: date.truncate(t: now(), unit: 1d))
  |> filter(fn: (r) => r["phase"] == "kwh")
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["description"] == "sum")
  |> filter(fn: (r) => r["location"] == "ac_indoor_unit" or r["location"] == "ac_outdoor_unit")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> keep(columns: ["_time", "_value", "location"])
'''

# CSV 변환
df_power_usage = query_to_dataframe(client, query_power_usage_data)
save_csv(df_power_usage, "%m_%d_power_usage_data.csv", "power_usage/")
print(df_power_usage.head())

                  time  value place        location device
0  2024-04-30 00:02:00   38.9  None  ac_indoor_unit   None
1  2024-04-30 00:04:00   38.9  None  ac_indoor_unit   None
2  2024-04-30 00:06:00   38.9  None  ac_indoor_unit   None
3  2024-04-30 00:08:00   38.9  None  ac_indoor_unit   None
4  2024-04-30 00:10:00   38.9  None  ac_indoor_unit   None
